In [1]:
#
# Run the hbv parameter estimation with full model
# https://github.com/rawlings-group/paresto/blob/master/examples/green_book/hbv_det.m
#

In [2]:
from kipet import KipetModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import t
from scipy.linalg import eig
import scipy.linalg as sla

    Set objects from pyomo.core.base.set or pyomo.core.  (deprecated in 5.7)
    (called from <frozen importlib._bootstrap>:219)
_SetProduct not found


ModuleNotFoundError: No module named 'kipet.library.top_level.clean'

In [ ]:
filename = '/home/paperspace/learn_kipet/kipet_paresto/data_sets/hbv_data.csv'
C_data = pd.read_csv(filename, index_col = 0)
C_data.shape

In [ ]:
C_data.head()

In [ ]:
# True parameters
x = np.array([2, 0.025, 1000, 0.25, 1.9985, 7.5E-6])
np.log10(x)

In [ ]:
kipet_model = KipetModel()

r1 = kipet_model.new_reaction('rxn1')

r1.add_component('A', state = 'concentration', init = 1.0)
r1.add_component('B', state = 'concentration', init = 0.0)
r1.add_component('C', state = 'concentration', init = 0.0)

r1.add_parameter('lk1',init = 0.8, bounds=(-6.0, 4.0))
r1.add_parameter('lk2',init = -1.1, bounds=(-6.0, 4.0))
r1.add_parameter('lk3',init = 3.1, bounds=(-6.0, 4.0))
r1.add_parameter('lk4',init = -0.77, bounds=(-6.0, 4.0))
r1.add_parameter('lk5',init = -0.16, bounds=(-6.0, 4.0))
r1.add_parameter('lk6',init = -5.46, bounds=(-6.0, 4.0))


In [ ]:
# define explicit system of ODEs
def rule_odes(m,t):
    exprs = dict()
    exprs['A'] = 10**m.P['lk2']*m.Z[t,'B'] - 10**m.P['lk4']*m.Z[t,'A']
    exprs['B'] = 10**m.P['lk1']*m.Z[t,'A']-10**m.P['lk2']*m.Z[t,'B']-10**m.P['lk6']*m.Z[t,'B']*m.Z[t,'C']
    exprs['C'] = 10**m.P['lk3']*m.Z[t,'A']-10**m.P['lk5']*m.Z[t,'C']-10**m.P['lk6']*m.Z[t,'B']*m.Z[t,'C']
    return exprs

r1.add_equations(rule_odes)
r1.set_times(0.0,100.0)

In [ ]:
r1.add_dataset('C_data', category = 'concentration', data = C_data)

In [ ]:
# sigmas as 1/wts used in the book
r1.variances = {'A':1, 'B':100, 'C':1e4}
r1.settings.collocation.nfe = 50
r1.settings.collocation.ncp = 3
r1.settings.collocation.scheme = 'LAGRANGE-RADAU'
r1.settings.parameter_estimator['solver'] = 'k_aug'
r1.settings.solver.linear_solver = 'ma27'

In [ ]:
r1.create_pyomo_model()
r1.model.P.pprint()

In [ ]:
r1.run_opt()

In [ ]:
print("The estimated parameters are:")
r1.results.show_parameters

In [ ]:
{k: 10**x for (k, x) in r1.results.P.items()}

In [ ]:
fig, ax = plt.subplots(3)
cmplist = ['A', 'B', 'C']
for (i, c) in enumerate(cmplist):
    ax[i].scatter(r1.results.Cm.index, r1.results.Cm[c])
    ax[i].plot(r1.results.Z.index, r1.results.Z[c])
    ax[i].set_xlabel('time')
    ax[i].set_ylabel(c)

In [ ]:
# since kipet outputs reduced hessian which equals covariance if right sigma values are specified. 
# since we just specified sigma=1, we need to adjust the covariance matrix output from KIPET to get the 
# estimated covariance matrix
mse = r1.results.objective / (C_data.shape[0]*3 - 6)
cov_est = 2 * mse * r1.results.parameter_covariance
cov_est

In [ ]:
eigval, eigvec = sla.eig(cov_est)
eigval, eigvec

In [ ]:
eigvec[:,4]

In [ ]:
dof = C_data.shape[0]*3 - 6
conf_delta = t.ppf(0.975, dof) * np.sqrt(np.diag(cov_est))
conf_delta, conf_delta / np.abs(np.array(list(r1.results.P.values())))